In [ ]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

import minsearch
import os
import json
import requests 
#
import openai
from openai import OpenAI
#


docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()
openai_key=os.environ['OPENAI_API_KEY'] 

In [ ]:
# #reading from a file..
# with open('documents.json','rt') as f_in:
#     docs_raw = json.load(f_in)

In [ ]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

#documents[:3]
user_query=[]
user_query.append('the course has already started, can I still enroll?')
user_query.append('how do I install kafka?')



In [ ]:
user_query

In [ ]:
def build_prompt(query, search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
    #get the context from 
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    #print(prompt)    
    return prompt



In [ ]:

def minisearch(query):
    
    #not best place maybe
    index = minsearch.Index(
        text_fields=["question","text","section"],
        keyword_fields=['course']
    )
    index.fit(documents)

    #higher weight for question field
    boost = {'question': 3.0}
    results = index.search(
        query=user_query,
        #filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    #results[0]
    return results


results_dbsearch = minisearch(user_query)
results_dbsearch

In [ ]:
def openai(query,engine='gpt-3.5-turbo'):

    try:
        client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        completion = client.chat.completions.create(
            model=engine,
            messages=[{"role":"user","content":query}]
        )    
    except Error as e:
        if e.status == 429:  # HTTP status code for Rate Limit Exceeded
            print("Rate limit exceeded. Please wait and retry.")
        else:
            print(f"Error: {e}")


    result = completion.choices[0].message.content

    return result

In [ ]:
results = openai(user_query)
results

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

#create the context based on the db results, pass along with the prompt
context = ""
for doc in results_dbsearch:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=user_query, context=context).strip()
prompt

In [ ]:
context

In [ ]:
prompt = build_prompt(user_query,results_dbsearch)
prompt

In [ ]:
results = openai(prompt)

In [ ]:
results

In [ ]:
def rag(query):
    search_results = minisearch(query)
    prompt = build_prompt(query,search_results)
    answer = openai(prompt)
    return answer

In [ ]:
rag('how do i install kafka?')

In [ ]:
rag('the course has already started, can I still enroll?')